In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import math
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [2]:
df1 = pd.read_csv('takehome_user_engagement.csv', encoding='ISO-8859-1')
df2 = pd.read_csv('takehome_users.csv', encoding='ISO-8859-1')

In [3]:
df1.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
df2.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [7]:
print(df1.shape)
print(df2.shape)

(207917, 3)
(12000, 10)


In [8]:
#Convert creation_time to datetime
df2['creation_time']=pd.to_datetime(df2.creation_time)

In [9]:
#Convert last_session_creation_time
lsct2=[]

for i in df2['object_id']:
    #If nan, use the last known login time. If last login is nan use creation time
    if math.isnan(df2[df2['object_id']==i]['last_session_creation_time']):
        if df1[df1['user_id']==i].time_stamp.empty:
            lsct2.append(df2[df2['object_id']==i]['creation_time'].iloc[0])
        else:
            lsct2.append(max(df1[df1['user_id']==i].time_stamp))
    else:
        lsct2.append(datetime.fromtimestamp(df2[df2['object_id']==i]['last_session_creation_time']))

df2['last_session_creation_time']=lsct2

#Use Id zero to indicate that a user was not invited by another user
df2['invited_by_user_id']=df2['invited_by_user_id'].fillna(0)
    
df2.info()

<ipython-input-9-42079b65d94f>:12: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  lsct2.append(datetime.fromtimestamp(df2[df2['object_id']==i]['last_session_creation_time']))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  12000 non-null  datetime64[ns]
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          12000 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(4), object(3)
memory usage: 937.6+ KB


In [10]:
df2.head(10)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 22:53:30,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 18:14:52,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 03:09:28,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 04:14:20,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,2013-12-18 21:37:06,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,2012-12-20 07:24:32,0,1,37,0.0
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,2013-07-31 05:34:02,1,1,74,0.0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,2013-11-05 04:04:24,0,0,302,0.0
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,2014-06-03 17:08:03,1,1,318,4143.0


In [11]:
df = pd.merge(df1,df2,'left',left_on='user_id', right_on='object_id')
df.head()

,time_stamp,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,2014-04-22 03:53:30,1,1,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 22:53:30,1,0,11,10803.0
1,2013-11-15 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0
2,2013-11-29 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0
3,2013-12-09 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0
4,2013-12-25 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0


In [12]:
ts = []
for i in df['time_stamp']:
    ts.append(datetime.strptime(i, '%Y-%m-%d %H:%M:%S').date())
df['ts_date'] = ts

In [13]:
#Users who meet the adoptions criteria

diff = timedelta(days=6)

users=[]
for i in df2['object_id']:
    if len(df[df.user_id == i].ts_date.unique()) >= 3:
        for j in df[df.user_id == i].ts_date.unique():
            f_day = j - diff
            count = 0
            for k in df[df.user_id == i].ts_date.unique():
                if k < j and k > f_day:
                    count=count+1
                if count >=3:
                    break
            if count >= 3:
                users.append(i)
                break
            


In [14]:
adopt = []
for i in df2['object_id']:
    if i in users:
        adopt.append(1)
    else:
        adopt.append(0)
df2['adopted_user'] = adopt

In [15]:
df2.index=df2['object_id']

In [16]:
df2.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
object_id,,,,,,,,,,,
1,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 22:53:30,1,0,11,10803.0,0
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 22:45:04,0,0,1,316.0,0
3,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 18:14:52,0,0,94,1525.0,0
4,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 03:09:28,0,0,1,5151.0,0
5,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 04:14:20,0,0,193,5240.0,0


In [17]:
X_cat = df2[['object_id','name','email','creation_source','opted_in_to_mailing_list',
            'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id']]
X_cont = df2[['creation_time','last_session_creation_time']]

In [18]:
def NormalizeDatetime(data):
    data = pd.to_datetime(data).astype('int64')
    max_a = data.max()
    min_a = data.min()
    min_norm = -1
    max_norm =1
    data = (data- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm
    return data

In [19]:
#Normalize datetimes
X_cont_norm=pd.DataFrame()
X_cont_norm['creation_time']=NormalizeDatetime(df2.creation_time)
X_cont_norm['last_session_creation_time']=NormalizeDatetime(df2.last_session_creation_time)

In [20]:
X_cont_norm.head()

,creation_time,last_session_creation_time
object_id,,
1,0.893592,0.876508
2,0.460681,0.816732
3,-0.197395,-0.205132
4,-0.026508,-0.032993
5,-0.366011,-0.358835


In [21]:
le=LabelEncoder()
X_cat_l=pd.DataFrame()

for i in X_cat:
    X_cat_l[i]=le.fit_transform(X_cat[i])
X_cat_l

,object_id,name,email,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,0,2087,1047,0,1,0,11,2325
1,1,8488,7290,1,0,0,1,56
2,2,1259,7681,1,0,0,94,298
3,3,2099,7971,0,0,0,1,1104
4,4,8708,3718,0,0,0,193,1127
...,...,...,...,...,...,...,...,...
11995,11995,7156,9573,1,0,0,89,1802
11996,11996,3521,535,4,0,0,200,0
11997,11997,4598,4322,0,1,1,83,1764
11998,11998,3261,11341,2,0,0,6,0


In [22]:
X_combine = pd.merge(X_cat_l,X_cont_norm,how='left', left_index=True,right_index=True)
X_combine.head()

,object_id,name,email,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_time,last_session_creation_time
0,0,2087,1047,0,1,0,11,2325,NaN,NaN
1,1,8488,7290,1,0,0,1,56,0.893592,0.876508
2,2,1259,7681,1,0,0,94,298,0.460681,0.816732
3,3,2099,7971,0,0,0,1,1104,-0.197395,-0.205132
4,4,8708,3718,0,0,0,193,1127,-0.026508,-0.032993


In [23]:
X = X_combine
y = df2['adopted_user']

In [24]:
df_enc = pd.merge(X_combine,y,how='left', left_index=True,right_index=True)

In [25]:
np.abs(df_enc.corr()['adopted_user']).sort_values(ascending= False)

adopted_user                  1.000000
last_session_creation_time    0.362165
creation_time                 0.119646
enabled_for_marketing_drip    0.011202
org_id                        0.010239
email                         0.007503
opted_in_to_mailing_list      0.006374
invited_by_user_id            0.006018
creation_source               0.005177
object_id                     0.000491
name                          0.000229
Name: adopted_user, dtype: float64

In [26]:
data = pd.merge(pd.merge(X_cat,X_cont,how='left', left_index=True,right_index=True),y,how='left', left_index=True,right_index=True)
data.head()

,object_id,name,email,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_time,last_session_creation_time,adopted_user
object_id,,,,,,,,,,,
1,1,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1,0,11,10803.0,2014-04-22 03:53:30,2014-04-21 22:53:30,0
2,2,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,0,0,1,316.0,2013-11-15 03:45:04,2014-03-30 22:45:04,0
3,3,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,0,0,94,1525.0,2013-03-19 23:14:52,2013-03-19 18:14:52,0
4,4,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,0,0,1,5151.0,2013-05-21 08:09:28,2013-05-22 03:09:28,0
5,5,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,0,0,193,5240.0,2013-01-17 10:14:20,2013-01-22 04:14:20,0
